In [1]:
import pandas as pd
from datetime import date, timedelta


In [64]:
flow_df_banff = pd.read_csv('bow-river-level-at-banff.csv',parse_dates=[['Date', 'Time']])
flow_df_calgary = pd.read_csv('bow-river-flow-target-feature.csv', parse_dates=[['Date', 'Time']])
elbow_precip = pd.read_csv('elbow-ranger-station-precip.csv', parse_dates=[['Date', 'Time']])
elbow_bragg_creek = pd.read_csv('elbow-river-at-bragg-creek.csv', parse_dates=[['Date', 'Time']])
elbow_glenmore_dam = pd.read_csv('elbow-river-flow-below-glenmore-dam-target-feature.csv', parse_dates=[['Date', 'Time']])
snow_pillow_elbow = pd.read_csv('snow-pillow-elbow.csv', parse_dates=[['Date', 'Time']])
lake_louise_precip = pd.read_csv('precip-at-lake-louise.csv', parse_dates=[['Date', 'Time']])
lake_louise_snow_pillow = pd.read_csv('snow-pillow-lake-louise.csv', parse_dates=[['Date', 'Time']])

In [59]:
def format_data(df):
    df= df.resample('60min', on='Date_Time').mean()
    df = df[df.index.month > 3]
    df = df[df.index.month < 10]
    df = df[df.index.year >1999]
    df = df[df.index.year <2020]
    df_parsed = df.fillna(method='backfill')
    return df_parsed

In [65]:
flow_df_calgary_parsed = format_data(flow_df_calgary)
flow_df_banff_parsed = format_data(flow_df_banff)
elbow_precip_parsed = format_data(elbow_precip)
elbow_bragg_creek_parsed = format_data(elbow_bragg_creek)
elbow_glenmore_dam_parsed = format_data(elbow_glenmore_dam)
snow_pillow_elbow_parsed = format_data(snow_pillow_elbow)
lake_louise_precip_parsed = format_data(lake_louise_precip)
lake_louise_snow_pillow_parsed = format_data(lake_louise_snow_pillow)

In [66]:
flow_df_calgary_parsed = flow_df_calgary_parsed.rename(columns={'Value(m3/s)':'calgary_bow_flow(m3/s)'})
flow_df_banff_parsed = flow_df_banff_parsed.rename(columns={'Value(m3/s)':'banff_bow_flow(m3/s)'})
elbow_precip_parsed = elbow_precip_parsed.rename(columns={'Value(mm)': 'elbow_precip(mm)'})

In [62]:
df = pd.merge(flow_df_calgary_parsed, flow_df_banff_parsed, left_index=True, right_index=True)
df = pd.merge(df, elbow_precip_parsed, left_index=True, right_index=True)
df = pd.merge(df, elbow_bragg_creek_parsed, left_index=True, right_index=True)
df = pd.merge(df, elbow_glenmore_dam_parsed, left_index=True, right_index=True)
df = pd.merge(df, snow_pillow_elbow_parsed, left_index=True, right_index=True)
df = pd.merge(df,lake_louise_precip_parsed, left_index=True, right_index=True)
df = pd.merge(df,lake_louise_snow_pillow_parsed, left_index=True, right_index=True)

In [68]:
dataframe2 = df

In [41]:
flow_df_calgary_parsed.to_pickle('../parsed/dataframe1.pkl')

In [69]:
dataframe2.to_pickle('../parsed/dataframe2.pkl')